# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f639ea72f70>
├── 'a' --> tensor([[-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570]])
└── 'x' --> <FastTreeValue 0x7f639ea72b80>
    └── 'c' --> tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                        [-1.7001, -0.1163,  2.1946, -0.4024],
                        [ 0.2964, -1.1194, -0.7314, -2.1851]])

In [4]:
t.a

tensor([[-0.9464,  0.9640,  0.8989],
        [ 0.4228,  0.6007, -0.1570]])

In [5]:
%timeit t.a

65.1 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f639ea72f70>
├── 'a' --> tensor([[-1.0980, -0.1474, -0.5716],
│                   [ 0.3070, -0.3898, -0.4694]])
└── 'x' --> <FastTreeValue 0x7f639ea72b80>
    └── 'c' --> tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                        [-1.7001, -0.1163,  2.1946, -0.4024],
                        [ 0.2964, -1.1194, -0.7314, -2.1851]])

In [7]:
%timeit t.a = new_value

58.6 ns ± 0.0412 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570]]),
    x: Batch(
           c: tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                      [-1.7001, -0.1163,  2.1946, -0.4024],
                      [ 0.2964, -1.1194, -0.7314, -2.1851]]),
       ),
)

In [10]:
b.a

tensor([[-0.9464,  0.9640,  0.8989],
        [ 0.4228,  0.6007, -0.1570]])

In [11]:
%timeit b.a

55.3 ns ± 0.035 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4951,  0.2849,  1.1496],
               [ 0.5048, -0.3925, -0.3596]]),
    x: Batch(
           c: tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                      [-1.7001, -0.1163,  2.1946, -0.4024],
                      [ 0.2964, -1.1194, -0.7314, -2.1851]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.156 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 487 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 621 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f62ef653940>
├── 'a' --> tensor([[[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]],
│           
│                   [[-0.9464,  0.9640,  0.8989],
│                    [ 0.4228,  0.6007, -0.1570]]])
└── 'x' --> <FastTreeValue 0x7f62ef653d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 645 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f639ea72be0>
├── 'a' --> tensor([[-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570],
│                   [-0.9464,  0.9640,  0.8989],
│                   [ 0.4228,  0.6007, -0.1570]])
└── 'x' --> <FastTreeValue 0x7f639ea72130>
    └── 'c' --> tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                        [-1.7001, -0.1163,  2.1946, -0.4024],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 53.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 68 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]],
       
               [[-0.9464,  0.9640,  0.8989],
                [ 0.4228,  0.6007, -0.1570]]]),
    x: Batch(
           c: tensor([[[ 0.4176, -0.1874, -0.3293,  1.0707],
                       [-1.7001, -0.1163,  2.1946, -0.4024],
                       [ 0.2964, -1.1194, -0.7314, -2.1851]],
         

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 65.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570],
               [-0.9464,  0.9640,  0.8989],
               [ 0.4228,  0.6007, -0.1570]]),
    x: Batch(
           c: tensor([[ 0.4176, -0.1874, -0.3293,  1.0707],
                      [-1.7001, -0.1163,  2.1946, -0.4024],
                      [ 0.2964, -1.1194, -0.7314, -2.1851],
                      [ 0.4176, -0.1874, -0.3293,  1.0707],
                      [-1.7001, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 569 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 4.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
